In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
import sklearn

plt.style.use('seaborn')

# Datos en las columnas
# ------------------------------
Edad (discreta), Sexo (categorica), Tipo de dolor de pecho (4 posibles dolores, categorica), Presion sanguinea (continua),
Colesterol (mg/dl) (continua), azucar en sangre mayor a 120mg/dl (categorica), resultado electrocardiograma (categorica), frecuencia cardiaca maxima (continua), angina inducida por ejercicio (categorica), st depression ? (continua), slope (categorica), radioscopia (categorica), thal (categorica 3 valores), enfermedad del corazon (categorica - target -> modelo de clasificacion)

In [2]:
# carga csv

heart_csv = pd.read_csv("heart.csv")
heart_csv.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
heart_csv['target'] = heart_csv['target'].astype('category')
heart_csv['sex'] = heart_csv['sex'].astype('category')
heart_csv['cp'] = heart_csv['cp'].astype('category')
heart_csv['fbs'] = heart_csv['fbs'].astype('category')
heart_csv['restecg'] = heart_csv['restecg'].astype('category')
heart_csv['fbs'] = heart_csv['fbs'].astype('category')
heart_csv['exang'] = heart_csv['exang'].astype('category')
heart_csv['slope'] = heart_csv['slope'].astype('category')
heart_csv['ca'] = heart_csv['ca'].astype('category')
heart_csv['thal'] = heart_csv['thal'].astype('category')

heart_csv.dtypes

age            int64
sex         category
cp          category
trestbps       int64
chol           int64
fbs         category
restecg     category
thalach        int64
exang       category
oldpeak      float64
slope       category
ca          category
thal        category
target      category
dtype: object

In [ ]:
# voy a mapear el target asi los graficos quedan mas explicitos
# o no lo voy a hacer 

#heart_csv['target'] = heart_csv['target'].map(lambda x: 'Si' if x == 1 else 'No')
#heart_csv.rename(columns = {'target' : 'Enfermo'}, inplace=True)

In [ ]:
# pairplot de las variables mas llamativas
sns.pairplot(heart_csv, vars=['age', 'trestbps', 'chol', 'thalach', 'oldpeak'], hue='target', diag_kws={'bw': 1})

In [ ]:
# violin plots 

fig, axs = plt.subplots(ncols=2, nrows=7, figsize=(16, 16), sharex='row')
ax = iter(tuple(x for sublist in axs.tolist() for x in sublist))
violins = lambda x: (sns.violinplot(y=heart_csv[x], x=heart_csv['target'], ax = next(ax), palette='mako') if x != "target" else None)
res = map(violins, list(heart_csv.columns))
a = list(res)
fig = plt.gcf()
fig.delaxes(axs[6][1])
plt.savefig("hola2.png")

In [ ]:
# matriz de correlaciones 

corr_matrix = heart_csv.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

# Modelo 1 - Regresion Logistica

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# para las variables categoricas (no las binarias) necesitamos alguna manera de encodearlas
# one-hot encoding o dummy variables (ONE HOT NUNCA PARA ESTOS MODELOS PQ LA MATRIZ ES SINGULAR)

# corazon tonto 
dummy_heart = pd.get_dummies(heart_csv, columns=['cp', 'restecg', 'slope', 'ca', 'thal'], drop_first=True)

# split 70 / 30
X_train, X_test, y_train, y_test = train_test_split(dummy_heart.drop('target', axis=1), dummy_heart['target'], test_size = 0.3, random_state=23)


X_train.head(5)

,age,sex,trestbps,chol,fbs,thalach,exang,oldpeak,cp_1,cp_2,...,restecg_2,slope_1,slope_2,ca_1,ca_2,ca_3,ca_4,thal_1,thal_2,thal_3
163,38,1,138,175,0,173,0,0.0,0,1,...,0,0,1,0,0,0,1,0,1,0
260,66,0,178,228,1,165,1,1.0,0,0,...,0,1,0,0,1,0,0,0,0,1
124,39,0,94,199,0,179,0,0.0,0,1,...,0,0,1,0,0,0,0,0,1,0
110,64,0,180,325,0,154,1,0.0,0,0,...,0,0,1,0,0,0,0,0,1,0
13,64,1,110,211,0,144,1,1.8,0,0,...,0,1,0,0,0,0,0,0,1,0


In [ ]:
# Veintitres

# primer modelo sin penalizacion  
# probaremos penalizacion l1 (| |), l2 (| |^2), elastic net (both of 'em)
lr = LogisticRegression(random_state=23, penalty='none', C=1e10)

# fiteamos la regresion logistica
lr.fit(X_train, y_train)

predicciones = lr.predict(X_test)

# el score en este caso es el accuracy (el loss es cross entropy...)
score = (predicciones == y_test).sum() / predicciones.size

print('Accuracy Regresion Logística: {}.\n'.format(score))

print(lr.coef_)

In [7]:
# quiero usar otra libreria que me de un poco mas de rigurosidad estadistica (aunque scikit-learn es muy buena para casi todo)

import statsmodels.api as sm

# la columna ca_4 hace que la convergencia del modelo sea pesima, la dropeamos momentaneamente (son todos 0's menos unos 5 1's)

X_train = X_train.drop('ca_4', axis=1)
X_test = X_test.drop('ca_4', axis=1)


logit_stats = sm.Logit(exog=np.asarray(X_train).astype('float'), endog=y_train.astype('float')).fit()

Optimization terminated successfully.
         Current function value: 0.306390
         Iterations 8


In [10]:
# ligeramente inferior al de scikit learn (pero tenemos un bello summary)
# falta agregar regularizacion!
probabilidades_estimadas = logit_stats.predict(np.asarray(X_test).astype('float'))

predicciones = list(map(round, probabilidades_estimadas))
acc = (predicciones == y_test).sum() / y_test.size

print('Accuracy Regresion Logística (statsmodels): {}.\n'.format(acc))
logit_stats.summary2()

Accuracy Regresion Logística (statsmodels): 0.8681318681318682.



<class 'statsmodels.iolib.summary2.Summary'>
"""
                         Results: Logit
=================================================================
Model:              Logit            Pseudo R-squared: 0.554     
Dependent Variable: target           AIC:              171.9092  
Date:               2021-02-26 00:02 BIC:              242.3975  
No. Observations:   212              Log-Likelihood:   -64.955   
Df Model:           20               LL-Null:          -145.59   
Df Residuals:       191              LLR p-value:      4.2829e-24
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     8.0000                                       
-------------------------------------------------------------------
           Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
-------------------------------------------------------------------
x1         0.0524     0.0305    1.7168   0.0860   -0.0074    0.1123
x2        -1.6270     0.7370   -2.2075   0.0273   -3.0715   -0.1824
x3        -0.0340     0.0136   -2.4943   0.0126   -0.0607   -0.0073
x4        -0.0024     0.0053   -0.4631   0.6433   -0.0128    0.0079
x5         0.4597     0.7139    0.6439   0.5197   -0.9396    1.8590
x6         0.0204     0.0153    1.3289   0.1839   -0.0097    0.0504
x7        -1.0006     0.5366   -1.8647   0.0622   -2.0523    0.0511
x8        -0.2349     0.2833   -0.8294   0.4069   -0.7901    0.3202
x9         1.1612     0.7037    1.6502   0.0989   -0.2180    2.5404
x10        2.1064     0.6211    3.3916   0.0007    0.8892    3.3237
x11        2.2684     0.8479    2.6754   0.0075    0.6066    3.9301
x12        0.6754     0.4886    1.3822   0.1669   -0.2823    1.6330
x13       -0.8014     3.1045   -0.2581   0.7963   -6.8861    5.2834
x14       -0.6257     0.9258   -0.6758   0.4991   -2.4403    1.1889
x15        0.5717     1.0161    0.5627   0.5736   -1.4197    2.5632
x16       -1.7854     0.6398   -2.7903   0.0053   -3.0395   -0.5313
x17       -3.4360     0.9209   -3.7311   0.0002   -5.2410   -1.6311
x18       -2.6691     1.1738   -2.2738   0.0230   -4.9697   -0.3684
x19        0.8206     3.1209    0.2629   0.7926   -5.2964    6.9375
x20        1.2835     3.0238    0.4245   0.6712   -4.6431    7.2101
x21       -0.2866     3.0951   -0.0926   0.9262   -6.3529    5.7798
=================================================================

"""